In [18]:
import pandas as pd
import matplotlib.pyplot as plt

## Splitting Dataset

Splitting the dataset to `train` and `test`, which equal distribution of gender in each set. Further EDA and training will be constructed on the `train` before testing.

In [ ]:
df = pd.read_csv("./dataset/participants.tsv", sep="\t")

In [30]:
def stratified_split(group):
    males = group[group["Gender"] == "M"].sample(frac=1, random_state=42)  # Shuffle
    females = group[group["Gender"] == "F"].sample(frac=1, random_state=42)  # Shuffle

    num_males_train = int(len(males) * 0.8)
    num_females_train = int(len(females) * 0.8)

    males["Set"] = ["Train"] * num_males_train + ["Test"] * (len(males) - num_males_train)
    females["Set"] = ["Train"] * num_females_train + ["Test"] * (len(females) - num_females_train)

    return pd.concat([males, females])

In [ ]:
df_A = df[df["Group"] == "A"]
df_C = df[df["Group"] == "C"]
df_F = df[df["Group"] == "F"]

df_A = stratified_split(df_A)
df_C = stratified_split(df_C)
df_F = stratified_split(df_F)

df = pd.concat([df_A, df_C, df_F], ignore_index=True)

In [34]:
df.to_csv("./dataset/participants.tsv", sep="\t")